Replicating fastai caravana challenge notebook for datascience bowl


- Predict Multiclass
- Get boundary + interior - dilation(boundary)
- Watershed

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys 

In [ ]:
sys.path.append('../../fastai/')

In [ ]:
from utility.utils import *

In [ ]:
from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
#torch.cuda.set_device(1)

In [ ]:
PATH = Path('../data/ds_bowl_2018/')
paths = list(PATH.iterdir())
paths

In [ ]:
TRAIN_DN = 'stage1_train' 
TEST_DN = 'stage2_test_final'

In [ ]:
masks_csv = pd.read_csv(paths[0])
masks_csv.head()

In [ ]:
NUCLEI_ID = '58406ed8ef944831c413c3424dc2b07e59aef13eb1ff16acbb3402b38b5de0bd'

In [ ]:
list((PATH/TRAIN_DN).iterdir())[:5]

In [ ]:
NUCLEI_IDS = [str(TRAIN_DIR).split('/')[-1] for TRAIN_DIR in list((PATH/TRAIN_DN).iterdir())]

In [ ]:
Image.open(PATH/TRAIN_DN/f'{NUCLEI_ID}/images/{NUCLEI_ID}.png')

In [ ]:
Image.open(PATH/TRAIN_DN/f'{NUCLEI_ID}/multiclass_mask.png')

In [ ]:
ims = [open_image(PATH/TRAIN_DN/f'{NUCLEI_ID}/images/{NUCLEI_ID}.png')
       for NUCLEI_ID in NUCLEI_IDS[:16]]

In [ ]:
def show_img(im, figsize=None, ax=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return ax

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(9, 6))
for i,ax in enumerate(axes.flat): show_img(ims[i], ax=ax)
plt.tight_layout(pad=0.1)

In [ ]:
# TOTAL NUMBER OF IMAGES WITH MASK LABELS
n = len(list((PATH/TRAIN_DN).iterdir()))
n

### Data Prep

In [ ]:
TRAIN_DN = Path('../data/ds_bowl_2018/stage1_train/')
TEST_DN = Path('../data/ds_bowl_2018/stage2_test_final/')

In [ ]:
TRAIN_DIRS = list(TRAIN_DN.iterdir())
TEST_DIRS = list(TEST_DN.iterdir())

In [ ]:
class FilesDataset(BaseDataset):
    def __init__(self, fnames, transform, path):
        self.path,self.fnames = path,fnames
        super().__init__(transform)
    def get_sz(self): return self.transform.sz
    def get_x(self, i): return open_image(os.path.join(self.path, self.fnames[i]))
    def get_n(self): return len(self.fnames)

    def resize_imgs(self, targ, new_path):
        dest = resize_imgs(self.fnames, targ, self.path, new_path)
        return self.__class__(self.fnames, self.y, self.transform, dest)

    def denorm(self,arr):
        """Reverse the normalization done to a batch of images.

        Arguments:
            arr: of shape/size (N,3,sz,sz)
        """
        if type(arr) is not np.ndarray: arr = to_np(arr)
        if len(arr.shape)==3: arr = arr[None]
        return self.transform.denorm(np.rollaxis(arr,1,4))

In [ ]:
test_im = cv2.imread(str(TRAIN_DN/f'{NUCLEI_ID}/multiclass_mask.png'), cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.imshow((test_im == 215)*1)

In [ ]:
def get_multi_y(y, y_sz = None):
    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    multi_y = np.dstack([(y == 30)*1, # back
    (y == 110)*1, # fore
    (y == 215)*1]).transpose(2,0,1) # bound
    if y_sz is not None:
        multi_y = np.array([cv2.resize(y.astype(np.uint8), (y_sz, y_sz)) for y in multi_y])
    return multi_y.astype(np.float32)

class NucleiDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path, y_sz=256):
        self.y_sz = y_sz
        self.y=y
        super().__init__(fnames, transform, path)
    def get_y(self, i): return get_multi_y(self.y[i], self.y_sz)
    def get_c(self): return 0

In [ ]:
TRAIN_X = [str(x_name/'images'/x_name.name) + '.png' for x_name in TRAIN_DIRS]
TRAIN_Y = [str(x_name/'multiclass_mask.png') for x_name in TRAIN_DIRS]

In [ ]:
# we need to give dummy TEST_Y path - THIS IS A HACK NOT AN ABSOLUTE SOLUTION
TEST_X = [str(x_name/'images'/x_name.name) + '.png' for x_name in TEST_DIRS]
TEST_Y = np.random.choice(TRAIN_Y, len(TEST_X))

In [ ]:
VAL_X = [TRAIN_X[0]]
VAL_Y = [TRAIN_Y[0]]
TRN_X  = TRAIN_X
TRN_Y = TRAIN_Y

In [ ]:
len(TRN_X), len(TRN_Y), len(VAL_X), len(VAL_Y), len(TEST_X), len(TEST_Y)

In [ ]:
TRN_Y

### Model Data

Using Imagenet stats or others

In [ ]:
# set model function
PATH = '../data/'
f = resnet18
sz = 256
bs = 8

In [ ]:
#stats = [0, 1] # no normalize
#tfms = tfms_from_stats(stats, sz, crop_type=CropType.NO, tfm_y=None) # non-imagenet stats
tfms = tfms_from_model(f, sz, crop_type=CropType.NO, tfm_y=None) # imagenet stats
dataset = ImageData.get_ds(NucleiDataset, (TRN_X, TRN_Y), (VAL_X, VAL_Y), tfms=tfms, test= (TEST_X, TEST_Y), path=PATH)
md = ImageData(PATH, dataset, bs, num_workers=16, classes=None)
denorm = md.trn_ds.denorm

In [ ]:
x,y = next(iter(md.trn_dl))
x_np, y_np = to_np(x[0]), to_np(y[0])

In [ ]:
x_np.min(), x_np.max()

In [ ]:
open_image(TRN_X[0]).max()

In [ ]:
y_np.min(), y_np.max()

In [ ]:
plt.imshow(denorm(x)[0])

### Check Data

In [ ]:
x,y = next(iter(md.val_dl))

In [ ]:
x_np = to_np(x[0])

In [ ]:
y_np = to_np(y[0])

In [ ]:
plt.imshow(x_np[1])

In [ ]:
plt.imshow(y_np[2])

In [ ]:
x = next(iter(md.test_dl))

### Dyanmic UNET

In [ ]:
from fastai.models.unet import *

In [ ]:
# load defined model
def get_encoder(f, cut):
    base_model = (cut_model(f(True), cut))
    return nn.Sequential(*base_model)

### ENCODER: RESNET18

Has `extra_block` to output the original image size

Steps of creating a Dynamic Unet Model:

- Choose your encoder model or define it yourself (make sure it's downsampling as H, W -> H//2, W//2)
- Initialize DynamicUnet as m = DynamicUnet(encoder)
- In order to get the model to gpu set m = m.cuda()

In [ ]:
f = resnet18
cut, cut_lr = model_meta[f]
cut, cut_lr

In [ ]:
??cut_model

In [ ]:
??get_encoder

In [ ]:
encoder = get_encoder(f, cut)

In [ ]:
m = DynamicUnet(encoder)

In [ ]:
inp = torch.ones(1, 3, 256, 256)
out = m(V(inp))

In [ ]:
m.sfs_idxs

In [ ]:
out.size()

In [ ]:
inp.size()

In [ ]:
m

### ENCODER: RESNET50

Has `extra_block` to output the original image size

In [ ]:
f = resnet50
cut, cut_lr = model_meta[f]
cut, cut_lr

In [ ]:
encoder = get_encoder(f, cut)

In [ ]:
m = DynamicUnet(encoder)

In [ ]:
inp = torch.ones(1, 3, 256, 256)
out = m(V(inp))

In [ ]:
out.size()

In [ ]:
m

### ENCODER: VGG16 

Doesn't have `extra_block`

In [ ]:
f = vgg16
cut, cut_lr = model_meta[f]
cut, cut_lr

In [ ]:
encoder = get_encoder(f, 30)
m = DynamicUnet(encoder)

In [ ]:
inp = torch.ones(1, 3, 256, 256)
out = m(V(inp))

In [ ]:
out.size()

In [ ]:
m

### Training Definitions

In [ ]:
def mask_loss(pred,targ):
    return F.binary_cross_entropy_with_logits(pred[:,0],targ[...,0])

def mask_acc(pred,targ): return accuracy_multi(pred[:,0], targ[...,0], 0.)

def dice(pred, targs):
    m1 = (pred[:,0]>0).float()
    m2 = targs[...,0]
    return 2. * (m1*m2).sum() / (m1+m2).sum()

def multi_acc(logits, targets):
    bs, c, h, w = logits.size()
    out2 = logits.view(bs,c,h*w).transpose(2,1).contiguous()
    input_ = out2.view(bs*h*w,c)

    # target for cross entropy
    _, idx = torch.max(targets, 1)
    target = idx.view(-1)

    return sum(torch.max(input_, dim=1)[1] == target) / len(target)

In [ ]:
# since I am training on multiclass data, loss_fn will be different
class MulticlassBCELoss2d(nn.Module):
    """
    Weights for a single sample which is repeated along the batch
    Inputs:
        weight: weigth tensor for a single sample
    """
    def __init__(self):
        super(MulticlassBCELoss2d, self).__init__()
        
    def forward(self, logits, targets):
        # input for cross entropy
        bs, c, h, w = logits.size()
        out2 = logits.view(bs,c,h*w).transpose(2,1).contiguous()
        input_ = out2.view(bs*h*w,c)
        
        # target for cross entropy
        _, idx = torch.max(targets, 1)
        target = idx.view(-1)
        return F.cross_entropy(input_, target)

### Test Loss and Metric

In [ ]:
targ = torch.stack([torch.ones(3,3)*0, torch.ones(3,3)*0, torch.ones(3,3)], dim=0)[None, :]
inp = torch.stack([torch.ones(3,3)*0.3, torch.ones(3,3)*0.3, torch.ones(3,3)*0.4], dim=0)[None, :]

In [ ]:
# imagine all ground truth is class 2
# output logits are given
inp, targ

In [ ]:
loss = MulticlassBCELoss2d()

In [ ]:
loss(V(inp), V(targ))

In [ ]:
F.softmax(V(inp))

In [ ]:
# softmax(.., ..., 0.400) = (..., ..., 0.3559)
- np.log(0.3559)

In [ ]:
multi_acc(inp, targ)

### Put everything together + Training (ResNet18)

In [ ]:
torch.cuda.set_device(1)

#### 0) Define wrapper model class for Fast.ai Magic

In [ ]:
# Wrap everything nicely
class UpsampleModel():
    def __init__(self, model, cut_lr, name='upsample'):
        self.model,self.name, self.cut_lr = model, name, cut_lr

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.encoder), [self.cut_lr]))
        return lgs + [children(self.model)[1:]]

#### 1) Define Encoder

In [ ]:
f = vgg16
cut, cut_lr = model_meta[f]
cut, cut_lr

#### 2) Specify Cut and Init DynamicUnet

Put model to gpu if you like

In [ ]:
encoder = get_encoder(f, 30).cpu()
m = DynamicUnet(encoder.cpu()).cpu()

# create a dummy input - I couldn't find a work around for this to co-op with cpu - gpu tensor weight
# incosistencies so it's best to make a single forward pass ourselves if we want to use the gpu
# otherwise it will be fine on cpu
inp = torch.ones(1,3,256,256)
out = m(V(inp).cpu())

In [ ]:
out.size()

#### 3) Put to GPU (Optional) - Check Decoder Network

You will see `extra_block` for ResNet like architecture where downsample happens in first activation.

In [ ]:
m.cuda(1)

In [ ]:
for p in list(m.parameters()):
    print(p.get_device())

#### 4) Wrap DynamicUnet to be Fast.ai Ready

`cut_lr` will be used to group layers for freezing

In [ ]:
# specify layer groups pre-freezing
# cut_lr is experimental and heavily dependent on data you have, let's try 9
models = UpsampleModel(m, cut_lr=20)

#### 5) Create learn object

In [ ]:
learn = ConvLearner(md, models)
learn.opt_fn=optim.Adam
learn.crit = MulticlassBCELoss2d()
learn.metrics=[multi_acc]

#### 6) Check your layers to make

In [ ]:
learn.models.get_layer_groups(False)[1]

In [ ]:
# freeze first 3 conv bn relu
learn.freeze_to(1)

In [ ]:
learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.1
learn.fit(lr,1,cycle_len=16,use_clr_beta=(10,10, 0.85, 0.9))

In [ ]:
test_preds = learn.predict_dl(md.test_dl)

### INTERIOR + BOUNDARY - EROSION(BOUNDARY)

In [ ]:
argmax_preds = [np.argmax(pred, 0) for pred in test_preds]

### WATERSHED

In [ ]:
sample_pred = argmax_preds[0]

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('coins.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3) # DILATION(FORE + BOUND)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg) # FORE
unknown = cv2.subtract(sure_bg,sure_fg)

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [ ]:
np.unique(argmax_preds[0])

In [ ]:
test_szs = [open_image(fn).shape[:-1] for fn in md.test_ds.fnames]

In [ ]:
test_szs[0]

In [ ]:
resized_argmax_preds = [np.clip(cv2.resize(pred.astype(np.uint8), sz), 0, 2) for sz, pred in zip(test_szs, argmax_preds)]

In [ ]:
watershed_preds= []

for sample_pred in resized_argmax_preds:
    sure_back = np.uint8((sample_pred > 0))

    sure_fore = np.uint8(sample_pred == 1)

    unknown = cv2.subtract(sure_back,sure_fore)

    ret, markers = cv2.connectedComponents(sure_fore)

    markers = markers + 1

    markers[unknown==255] = 0

    watershed_preds.append(markers)


In [ ]:
def ws_to_rles(x, cutoff=0.5):
    """takes probability mask and yields for generator by looping over all labels"""
    lab_img = x
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [ ]:
def get_submission_df(preds, test_ds, rle_func=ws_to_rles):
    """
    Takes resized preds and ta
    est dataset
    to return rle df
    Inputs:
        preds (list): list of np.arrays which has 2d binary mask predictions
        test_ds (Dataset): test dataset
        rle_func (function): function to encode each binary mask prediction with run length encoding
    Return:
        sub (pd.dataframe): pandas dataframe for submission
    """
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ds.fnames):
        id_ = id_.split('/')[-2]
        rle = list(rle_func(preds[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))

    sub = pd.DataFrame()
    sub['ImageId'] = new_test_ids
    sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
    return sub

In [ ]:
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [ ]:
sub = get_submission_df(watershed_preds, md.test_ds)

In [ ]:
sub.to_csv('./final_sub.csv', index=False)

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink('./final_sub.csv')